In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
MONEY_FIELDS = 'donations_grants	sponsorship	registration_fees	travel_accommodation	fees	related_expenses	total'.split()
MONEY_FIELDS_ONLY = list(set(MONEY_FIELDS) - {'total'})
DOC_MONEY_FIELDS = list(set(MONEY_FIELDS_ONLY) - {'sponsorship', 'donations_grants'})
MONEY_FIELDS_DIRTY = ['%s_dirty' % x for x in MONEY_FIELDS_ONLY]

pd.set_option('display.float_format', lambda x: '{0:,.2f}'.format(x))
%matplotlib inline

In [20]:
df = pd.read_csv('aerztefranken.csv', encoding='utf-8')
df.head()

,company,name,name_unq,location,location_fix,address,donations_grants,sponsorship,registration_fees,travel_accommodation,fees,related_expenses,total,type
0,"AbbVie AG, Baar",Patrick Aepli,Patrick Aepli,Luzern,Luzern,Spitalstrasse,0.00,0.00,247.00,439.00,0.00,0.00,686.00,hcp
1,"AbbVie AG, Baar",Andre Aeschlimann,Andre Aeschlimann,Bad Zurzach,Bad Zurzach,Quellenstrasse 34,0.00,0.00,0.00,0.00,"1,000.00",0.00,"1,000.00",hcp
2,"AbbVie AG, Baar",Carlo Albani,Albani Carlo,Zürich,Zürich,Bahnhofplatz 5,0.00,0.00,93.00,277.00,0.00,0.00,370.00,hcp
3,"AbbVie AG, Baar",Andreas Altmann,Andreas Altmann,Riehen,Riehen,In den Neumatten 63,0.00,0.00,0.00,0.00,600.00,0.00,600.00,hcp
4,"AbbVie AG, Baar",Anca Antonino,Anca Antonino,Vevey,Vevey,Boulevard Paderewski 3,0.00,0.00,330.00,734.00,0.00,0.00,"1,064.00",hcp


In [22]:
df[MONEY_FIELDS_ONLY] = df[MONEY_FIELDS_ONLY].applymap(lambda x: x if pd.isnull(x) else (np.nan if x == 0.0 else x))
df['computed_total'] = df[MONEY_FIELDS_ONLY].sum(1)

0     686.00
1   1,000.00
2     370.00
3     600.00
4   1,064.00
Name: computed_total, dtype: float64

In [180]:
def company_aggregate(x):
    result = {}
    for m in MONEY_FIELDS:
        result[m+'_cnt'] = x[m].count()
        result[m+'_sum'] = x[m].sum()
    return pd.Series(result, name='metrics')
df.groupby(['company','type']).apply(company_aggregate)

metrics                                                  donations_grants_cnt  \
company                                            type                         
A Menarini AG, Zürich                              hco                  11.00   
                                                   hcp                   0.00   
AbbVie AG, Baar                                    hco                  30.00   
                                                   hcp                   0.00   
Actelion Pharma Schweiz AG, Baden                  hco                  23.00   
                                                   hcp                   0.00   
Alcon Switzerland SA, Rotkreuz                     hco                   0.00   
                                                   hcp                   0.00   
Allergan AG, Lachen                                hco                   2.00   
                                                   hcp                   0.00   
Almirall AG, Wallisellen                           hco                   0.00   
                                                   hcp                   0.00   
Amgen Switzerland AG, Zug                          hco                  19.00   
                                                   hcp                   0.00   
Astellas Pharma AG, Wallisellen                    hco                  14.00   
                                                   hcp                   0.00   
AstraZeneca AG, Zug                                hco                  38.00   
                                                   hcp                   0.00   
BGP Products GmbH, Baar                            hco                   0.00   
                                                   hcp                   0.00   
Baxalta Schweiz AG, Volketswil                     hco                   4.00   
                                                   hcp                   0.00   
Baxter AG, Volketswil                              hco                   9.00   
                                                   hcp                   0.00   
Bayer Schweiz AG, Zürich                           hco                   2.00   
                                                   hcp                   0.00   
Biogen Switzerland AG, Zug                         hco                   3.00   
                                                   hcp                   0.00   
Biotest (Schweiz) AG, Rupperswil                   hco                   1.00   
                                                   hcp                   0.00   
...                                                                       ...   
Novo Nordisk Pharma AG, Küsnacht                   hco                  14.00   
                                                   hcp                   0.00   
Orion Pharma AG, Zug                               hco                   0.00   
                                                   hcp                   0.00   
Otsuka Pharmaceutical (Switzerland) GmbH, Glatt... hco                   1.00   
                                                   hcp                   0.00   
Pfizer AG, Zürich                                  hco                  13.00   
                                                   hcp                   0.00   
Pierre Fabre Pharma AG, Allschwil                  hco                   5.00   
                                                   hcp                   0.00   
Roche Pharma (Schweiz) AG, Reinach BL              hco                  22.00   
                                                   hcp                   0.00   
Sandoz Pharmaceuticals AG, Rotkreuz                hco                   4.00   
                                                   hcp                   0.00   
Sanofi Pasteur MSD AG, Baar                        hco                   5.00   
                                                   hcp                   0.00   
Servier (Suisse) SA, Satigny   

In [133]:
# TODO: company_df?

pd.merge(
    pd.merge(
        df.groupby(['company','type']).apply(lambda x:
            x[MONEY_FIELDS_ONLY].sum()
        ),
        df.groupby(['company','type']).apply(lambda x: pd.Series({
            'count': x[MONEY_FIELDS_ONLY].count().sum()
        })),
        left_index=True, right_index=True
    ),
    df.groupby(['company','type']).apply(lambda x: pd.Series({
        'total': x['total'].sum()
    })),
    left_index=True, right_index=True
)

registration_fees  \
company                                            type                      
A Menarini AG, Zürich                              hco            1,812.00   
                                                   hcp           59,563.00   
AbbVie AG, Baar                                    hco            3,597.00   
                                                   hcp           87,498.00   
Actelion Pharma Schweiz AG, Baden                  hco            3,850.00   
                                                   hcp           32,532.20   
Alcon Switzerland SA, Rotkreuz                     hco                 nan   
                                                   hcp                 nan   
Allergan AG, Lachen                                hco                 nan   
                                                   hcp            2,908.00   
Almirall AG, Wallisellen                           hco                 nan   
                                                   hcp              415.80   
Amgen Switzerland AG, Zug                          hco           29,288.81   
                                                   hcp           56,953.23   
Astellas Pharma AG, Wallisellen                    hco           13,761.90   
                                                   hcp           23,211.30   
AstraZeneca AG, Zug                                hco           46,050.00   
                                                   hcp            3,795.70   
BGP Products GmbH, Baar                            hco                 nan   
                                                   hcp            6,883.00   
Baxalta Schweiz AG, Volketswil                     hco           10,944.90   
                                                   hcp               77.40   
Baxter AG, Volketswil                              hco              830.08   
                                                   hcp            6,381.22   
Bayer Schweiz AG, Zürich                           hco           11,221.55   
                                                   hcp           41,888.96   
Biogen Switzerland AG, Zug                         hco                 nan   
                                                   hcp            2,915.05   
Biotest (Schweiz) AG, Rupperswil                   hco                 nan   
                                                   hcp                 nan   
...                                                                    ...   
Novo Nordisk Pharma AG, Küsnacht                   hco            1,335.00   
                                                   hcp           66,721.00   
Orion Pharma AG, Zug                               hco                 nan   
                                                   hcp            9,901.24   
Otsuka Pharmaceutical (Switzerland) GmbH, Glatt... hco                 nan   
                                                   hcp            5,185.65   
Pfizer AG, Zürich                                  hco            3,265.00   
                                                   hcp           61,949.00   
Pierre Fabre Pharma AG, Allschwil                  hco            3,064.06   
                                                   hcp           11,778.87   
Roche Pharma (Schweiz) AG, Reinach BL              hco            3,187.00   
                                                   hcp           71,690.00   
Sandoz Pharmaceuticals AG, Rotkreuz                hco            3,016.00   
                                                   hcp           33,493.51   
Sanofi Pasteur MSD AG, Baar                        hco              422.78   
                                                   hcp            9,029.82   
Servier (Suisse) SA, Satigny                       hco            3,757.00   
                                                   hcp           19,342.00   
Stallergenes AG, Dietlikon                         hco                 nan   
 

In [6]:
agg_df = []
for company in df.company.unique():
    
    agg_df.append({
            'company': company,
            'year': 2015
            'type': hcphco,
            'label': field,
            'count': count,
            'amount': amount
        })

array(['AbbVie AG, Baar', 'Actelion Pharma Schweiz AG, Baden',
       'Alcon Switzerland SA, Rotkreuz', 'Allergan AG, Lachen',
       'Almirall AG, Wallisellen', 'A Menarini AG, Zürich',
       'Amgen Switzerland AG, Zug', 'Astellas Pharma AG, Wallisellen',
       'AstraZeneca AG, Zug', 'Baxalta Schweiz AG, Volketswil',
       'Baxter AG, Volketswil', 'Bayer Schweiz AG, Zürich',
       'BGP Products GmbH, Baar', 'Biogen Switzerland AG, Zug',
       'Biotest (Schweiz) AG, Rupperswil',
       'Boehringer Ingelheim (Schweiz) GmbH, Basel',
       'Bristol-Myers Squibb GmbH, Cham', 'Celegene GmbH, Zürich',
       'CSL Behring AG, Bern', 'Daiichi Sankyo (Shweiz) AG, Thalwil',
       'Eisai Pharma AG, Zürich', 'Eli Lilly (Suisse) SA, Vernier',
       'Ferring AG, Baar', 'Future Health Pharma GmbH, Wetzikon',
       'Galderma Schweiz AG', 'Genzyme a Sanofi Company, Baar',
       'GILEAD Sciences Switzerland Sàrl, Zug',
       'GlaxoSmithKline AG, Münchenbuchsee',
       'Grünenthal Pharma AG, 